## Initialization of model, constants, paths etc.

In [6]:
import configparser
config = configparser.ConfigParser()
config.read('config.cfg')

dataset_storage_path = config['Mistral-SC']['dataset_storage_path']
model_storage_path = config['Mistral-SC']['model_storage_path']
output_path = config['Mistral-SC']['output_dir']
libcuda_path = config['Unsloth']['libcuda_path']
library_path = config['Unsloth']['library_path']

import torch
num_gpus = torch.cuda.device_count()
print(f"Number of GPUs available: {num_gpus}")
print(torch.cuda.mem_get_info())

import os
os.environ["TRITON_LIBCUDA_PATH"]=libcuda_path
os.environ["LIBRARY_PATH"]=library_path

from unsloth import FastLanguageModel
max_seq_length = 4096*2
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_storage_path, 
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

Number of GPUs available: 1
(27853062144, 47842000896)


In [ ]:
from datasets import load_dataset
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    #instructions = examples["instruction"]
    instruction = "Summarize the following text:"
    inputs       = examples["text"]
    outputs      = examples["abstract"]
    instructions = [instruction for _ in range(len(inputs))]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "instruction": instructions, "input": inputs, "output": outputs,  "text" : texts, }
pass

data_files = {"test":f"{dataset_storage_path}/sumeczech-1.0-test.jsonl"}
dataset = load_dataset("json", data_files=data_files)
test_dataset = dataset["test"]
test_dataset = test_dataset.map(formatting_prompts_func, batched = True).select_columns(["input", "output", "text", "instruction"])

## Generate summaries for evaluation

In [7]:
articles = []
for sample in test_dataset:
    formatted_sample = alpaca_prompt.format(
        "Summarize the following text:", # instruction
        sample["input"], # input
        "", # output - leave this blank for generation!
    )
    articles.append(formatted_sample)
import json
generated_outputs = []
with open(f"{output_path}/generated_outputs_temp.json", "r", encoding="utf-8") as readfile:
    x = json.loads(readfile.read())
    generated_outputs = x if isinstance(x, list) else []
already_processed_len = (len(generated_outputs))
for i, article in enumerate(articles[already_processed_len:]):
    inputs = tokenizer([article], truncation=True, max_length = max_seq_length, return_tensors = "pt").to("cuda")
    if inputs["input_ids"].size(1) >= max_seq_length:
        decoded = tokenizer.batch_decode(inputs["input_ids"])
        new_article = f"{decoded[0]}\n### Response:\n" 
        inputs = tokenizer([new_article], return_tensors = "pt").to("cuda")
        print(inputs["input_ids"].size(1))
    outputs = model.generate(**inputs, max_new_tokens = 512)
    result = tokenizer.batch_decode(outputs)

    #print(result)
    for res in result:
        splitted_res = res.split("### Response:")
        generated_outputs.append(splitted_res[1].split("</s>")[0].replace("\n",""))
        with open(f"{output_path}/generated_outputs_temp.json", "w", encoding="utf-8") as file:
            file.write(json.dumps(generated_outputs))
    print(f"Progress: {already_processed_len+i+1}/{len(articles)}")
#generated_outputs
generated_outputs_json = json.dumps(generated_outputs)
with open(f"{output_path}/generated_outputs.json", "w", encoding="utf-8") as file:
    file.write(generated_outputs_json)

## Evaluate (get rouge score of the model)

In [77]:
import json

import rouge_raw
eval = rouge_raw.RougeRaw()
abstract = dataset["test"]["abstract"]
roguerawscore = eval.corpus(gold=abstract, system=generated_outputs)

with open(f"{output_path}/eval_sumeczech.txt", "w") as file:
    file.write("ROUGE-1 F: " + str(roguerawscore["1"].f * 100) + "\n")
    file.write("ROUGE-1 P: " + str(roguerawscore["1"].p * 100) + "\n")
    file.write("ROUGE-1 R: " + str(roguerawscore["1"].r * 100) + "\n")

    file.write("ROUGE-2 F: " + str(roguerawscore["2"].f * 100) + "\n")
    file.write("ROUGE-2 P: " + str(roguerawscore["2"].p * 100) + "\n")
    file.write("ROUGE-2 R: " + str(roguerawscore["2"].r * 100) + "\n")

    file.write("ROUGE-L F: " + str(roguerawscore["L"].f * 100) + "\n")
    file.write("ROUGE-L P: " + str(roguerawscore["L"].p * 100) + "\n")
    file.write("ROUGE-L R: " + str(roguerawscore["L"].r * 100) + "\n")